## ライブラリインストール

In [16]:
import os
import random

import numpy as np
import pandas as pd
# import optuna
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, accuracy_score, precision_score, recall_score, f1_score, classification_report
from sklearn.model_selection import KFold, StratifiedKFold

import tensorflow as tf
from tensorflow.keras import datasets, layers, models
from tensorflow.keras.utils import plot_model, to_categorical
from tensorflow.keras.layers import Input, Dense, Conv2D, Activation
from tensorflow.keras.layers import MaxPooling2D, UpSampling2D, BatchNormalization, Dropout, GlobalAveragePooling2D
from tensorflow.keras.models import Model, Sequential
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.applications import EfficientNetB0, EfficientNetB1, ResNet50

In [2]:
def set_randvalue(value):
    # Set a seed value
    seed_value= value 
    # 1. Set `PYTHONHASHSEED` environment variable at a fixed value
    os.environ['PYTHONHASHSEED']=str(seed_value)
    # 2. Set `python` built-in pseudo-random generator at a fixed value
    random.seed(seed_value)
    # 3. Set `numpy` pseudo-random generator at a fixed value
    np.random.seed(seed_value)
    # 4. Set `tensorflow` pseudo-random generator at a fixed value
    tf.random.set_seed(seed_value)

seed_value = 42
set_randvalue(seed_value)

## CSVロード

In [3]:
# train.csvが元のラベルごとにディレクトリに保存されていたデータから作成したcsv
df = pd.read_csv("../data/input/csvs/train.csv")
df["label"] = df["label"].astype(str)
df.head(3)

,image,label
0,img_100026.jpg,0
1,img_10003.jpg,0
2,img_100050.jpg,0


## モデル作成

In [31]:
# EfficientNetB0のモデルを作成
def create_model(weight_flg=False):
    weight = None
    if weight_flg:
        weight = "../model/efficientnetb0_notop.h5" # ImageNetで学習されたモデルをロード
    # include_top=False; 全結合層なし
#     base_model = EfficientNetB0(weights=weight, include_top=False, pooling='avg', input_shape=(224,224,3))
    base_model = ResNet50(weights='imagenet', include_top=False, pooling='avg', input_shape=(224,224,3))
    x = Dense(512, activation='relu')(base_model.output)
    x = Dropout(0.5)(x)
    x = Dense(512, activation='relu')(x)
    +
    x = Dropout(0.5)(x)
    output = Dense(10, activation='softmax')(x)
    model = Model(base_model.input, output)
    
    model.compile(optimizer=Adam(), loss='categorical_crossentropy', metrics=['accuracy'])
    return model

## 学習

In [33]:
# Hyper parameters
img_size = 224
batch_size = 32
epochs = 250

In [14]:
# Generator作成
train_datagen = ImageDataGenerator(
    rescale=1./255,
    rotation_range=20, # rotation range
    width_shift_range=30/img_size, # 30 pixel
    height_shift_range=30/img_size, # 30 pixel
    zoom_range=0.2
)

valid_datagen = ImageDataGenerator(rescale=1./255)
test_datagen = ImageDataGenerator(rescale=1./255)

In [36]:
%%time
# 学習と評価データでのラベルの分布数を一定に保ち学習
skf = StratifiedKFold(n_splits = 5, shuffle=True, random_state=0)
splitter = skf.split(df["image"],df["label"])
for i, (train_ids, valid_ids) in enumerate(splitter, 1):
#     if i == 1: continue
    # データ生成
    train, valid = df.iloc[train_ids], df.iloc[valid_ids]
    train_datagenerator = train_datagen.flow_from_dataframe(
        train,
        directory='../data/input/imgs/train/imgs/',
        x_col='image',
        y_col='label',
        target_size=(img_size, img_size),
        class_mode='categorical',
        batch_size=batch_size,
        seed=seed_value
    )

    valid_datagenerator = valid_datagen.flow_from_dataframe(
        valid,
        directory='../data/input/imgs/train/imgs/',
        x_col='image',
        y_col='label',
        target_size=(img_size, img_size),
        class_mode='categorical',
        batch_size=batch_size,
        seed=seed_value
    )
    
    model = create_model(weight_flg=True)
    
    # 早期終了
    early_stop = EarlyStopping(monitor='val_loss', patience=3, verbose=1, mode='auto')
    
    # 早期終了するのでval_lossが小さいモデルを保存
    model_path = '../model/' +  'Train005_' + "fold" + str(i) + "_best_model.h5"
    checkpoint = ModelCheckpoint(
                    filepath=model_path,
                    monitor='val_loss',
                    save_best_only=True,
                    period=1)
    
    history = model.fit(
        train_datagenerator,
        steps_per_epoch=int(len(train)//batch_size),
        epochs=epochs,
        validation_data=valid_datagenerator,
        validation_steps=int(len(valid)//batch_size),
        verbose=1,
        shuffle=True,
        callbacks=[early_stop, checkpoint])

Found 17939 validated image filenames belonging to 10 classes.
Found 4485 validated image filenames belonging to 10 classes.
Epoch 1/250
560/560 [==============================] - 171s 305ms/step - loss: 1.9423 - accuracy: 0.2552 - val_loss: 2.4916 - val_accuracy: 0.0908
Epoch 2/250
560/560 [==============================] - 170s 304ms/step - loss: 0.7239 - accuracy: 0.7514 - val_loss: 3.4304 - val_accuracy: 0.4993
Epoch 3/250
560/560 [==============================] - 171s 305ms/step - loss: 0.3717 - accuracy: 0.8886 - val_loss: 0.5407 - val_accuracy: 0.8471
Epoch 4/250
560/560 [==============================] - 170s 304ms/step - loss: 0.2701 - accuracy: 0.9206 - val_loss: 0.3063 - val_accuracy: 0.9190
Epoch 5/250
560/560 [==============================] - 170s 304ms/step - loss: 0.2409 - accuracy: 0.9327 - val_loss: 0.3020 - val_accuracy: 0.9067
Epoch 6/250
560/560 [==============================] - 170s 303ms/step - loss: 0.1920 - accuracy: 0.9443 - val_loss: 0.8735 - val_accuracy: 

## 推論

In [37]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')

test_generator = test_datagen.flow_from_dataframe(
    submit,
    directory='../data/input/imgs/test/',
    x_col='img',
    y_col='c0', # ダミー変数
    target_size=(img_size, img_size),
    class_mode=None,
    batch_size=1,
    verbose=1,
    shuffle=False
)

columns = submit.columns.values
labels = submit.columns[1:].values

def inference(fold_num):
    model = create_model()
    weight_path = '../model/Train005_fold' + str(fold_num) + '_best_model.h5'
    output_path = '../data/output/Train005_fold' + str(fold_num) + '_sub.csv'
    model.load_weights(weight_path)
    
    pred = model.predict(test_generator, verbose=1)
    pred_df = pd.DataFrame(columns=columns)
    pred_df['img'] = submit['img']
    pred_df[labels] = pred
    pred_df.to_csv(output_path, index=False)
    
for i in range(1,6):
    inference(i)

Found 79726 validated image filenames.
79726/79726 [==============================] - 420s 5ms/step


#### アンサンブル：単純平均

In [38]:
submit = pd.read_csv('../data/input/csvs/sample_submission.csv')
columns = submit.columns.values
labels = submit.columns[1:].values

ensemble = 0
for i in range(1,6):
    path = "../data/output/Train005_fold" + str(i) +"_sub.csv"
    ensemble += pd.read_csv(path).values[:,1:] / 5 # fold数で割る

ensemble_df = pd.DataFrame(columns=columns)
ensemble_df['img'] = submit['img']
ensemble_df[labels] = ensemble

ensemble_df.to_csv("../data/output/Train005_ensemble_sub.csv", index=False)